<a href="https://colab.research.google.com/github/JKniaaa/Roster-Scheduler/blob/main/SchedulerModel_beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [282]:
!pip install pandas
!pip install ortools

In [283]:
import pandas as pd
from datetime import datetime
from collections import defaultdict

def load_nurse_profiles(path='nurse_profiles.xlsx'):
    df = pd.read_excel(path)
    df['Name'] = df['Name'].str.strip()
    return df

# def load_shift_preferences(path='nurse_preferences.xlsx'):
#     df = pd.read_excel(path, index_col=0)
#     df.columns = [pd.to_datetime(col).date() for col in df.columns]
#     return df

def load_shift_preferences(path='nurse_preferences.xlsx'):
    df = pd.read_excel(path)
    df.rename(columns={df.columns[0]: 'Name'}, inplace=True)
    df.set_index('Name', inplace=True)

    # Strip text like "Wed" or "Thurs" and parse dates
    cleaned_columns = [
        pd.to_datetime(col.strip().split()[-1], format="%Y-%m-%d").date()
        for col in df.columns
    ]
    df.columns = cleaned_columns

    df.index = df.index.str.strip()  # Ensure names are clean
    return df



In [284]:
def validate_nurse_data(profiles_df, preferences_df):
    profile_names = set(profiles_df['Name'].str.strip())
    preference_names = set(preferences_df.index.str.strip())

    missing_in_prefs = profile_names - preference_names
    extra_in_prefs = preference_names - profile_names

    if missing_in_prefs or extra_in_prefs:
        raise ValueError(
            f"Mismatch between nurse profiles and preferences:\n"
            f"❌ Missing in preferences: {missing_in_prefs}\n"
            f"❌ Extra in preferences: {extra_in_prefs}"
        )
    print("✅ Nurse profile and preference names match.")


In [285]:
from ortools.sat.python import cp_model
from datetime import timedelta
import pandas as pd

def build_schedule_model(profiles_df, preferences_df, start_date, num_days):
    """
    Builds a nurse schedule satisfying hard constraints A/B exactly,
    and relaxing all other hard constraints with a huge‐penalty slack
    so that there is always at least one solution.
    Returns a schedule DataFrame and a summary DataFrame.
    """

    # === Constants ===
    SHIFT_LABELS = ['AM','PM','Night']
    SHIFT_HOURS  = [7,7,10]
    DAYS_PER_WEEK = 7

    # Hard constraint params (A/B remain “real” hard below)
    MIN_NURSES_PER_SHIFT   = 4
    MIN_SENIORS_PER_SHIFT  = 1
    MAX_WEEKLY_HOURS       = 42
    MAX_MC_DAYS_PER_WEEK   = 2
    AM_COVERAGE_MIN_PERCENT = 60

    # Penalty weights for soft and relaxed constraints
    PEN_HUGE            = 1_000_000  # for relaxing everything except A/B
    PEN_UNDER4          =   500_000
    PEN_DOUBLE_SHIFT    =    31_250
    PEN_HOURS40         =   125_625
    PEN_WEEKEND_REST    =   250_000
    PEN_AM60            =    15_625
    PEN_AM60_SENIORS    =     7_813
    PEN_HOURS30         =    62_500
    PEN_FAIRNESS_GAP    =     3_906
    PEN_PREF_UNMET      =     1_953

    # === Model setup ===
    model = cp_model.CpModel()
    nurses      = profiles_df.to_dict('records')
    nurse_names = [n['Name'] for n in nurses]
    senior_names= {n['Name'] for n in nurses if n['Title']=='Senior'}
    shift_str_to_idx = {lbl:i for i,lbl in enumerate(SHIFT_LABELS)}

    # parse MC and preferences
    shift_prefs = {n:{} for n in nurse_names}
    mc_days     = {n:set() for n in nurse_names}
    for nurse,row in preferences_df.iterrows():
        for date,val in row.items():
            d = (date - start_date).days
            if pd.isna(val) or not (0<=d<num_days): continue
            v = val.strip().upper()
            if v=='MC':
                mc_days[nurse].add(d)
            elif v in shift_str_to_idx:
                shift_prefs[nurse][d] = shift_str_to_idx[v]

    # weekend pairs
    weekend_days = [(i,i+1) for i in range(num_days-1)
                    if (start_date+timedelta(days=i)).weekday()==5]

    # decision vars
    work = {}
    for n in nurse_names:
        for d in range(num_days):
            for s in range(3):
                work[n,d,s] = model.NewBoolVar(f"w_{n}_{d}_{s}")

    total_sat = {}
    penalty_terms = []

    # === A) MC days: no work (kept hard) ===
    for n in nurse_names:
        for d in mc_days[n]:
            for s in range(3):
                model.Add(work[n,d,s]==0)

    # === B) Max 2 MC per week & no 3 consecutive (kept hard) ===
    for n in nurse_names:
        # weekly count
        for w in range((num_days+6)//7):
            week = range(w*7, min((w+1)*7, num_days))
            if sum(1 for d in week if d in mc_days[n])>MAX_MC_DAYS_PER_WEEK:
                raise ValueError(f"{n} >{MAX_MC_DAYS_PER_WEEK} MCs in wk{w}")
        # no 3 consec
        sm = sorted(mc_days[n])
        for i in range(len(sm)-2):
            if sm[i+2]-sm[i]==2:
                raise ValueError(f"{n} 3 consec MC days")

    # === C) ≤ 2 shifts/day: *relaxed* with huge slack ===
    for n in nurse_names:
        for d in range(num_days):
            sw = sum(work[n,d,s] for s in range(3))
            slack = model.NewBoolVar(f"slack2sh_{n}_{d}")
            # either sw<=2 or slack=1
            model.Add(sw <= 2).OnlyEnforceIf(slack.Not())
            # if slack=1, skip the ≤2 rule
            penalty_terms.append(slack * PEN_HUGE)

    # === D) ≤ 42h/week hard, but ≥40 & ≥30 are soft ===
    for n in nurse_names:
        for w in range((num_days+6)//7):
            week = range(w*7, min((w+1)*7, num_days))
            hours = sum(work[n,d,s]*SHIFT_HOURS[s] for d in week for s in range(3))
            # keep ≤42h as hard:
            model.Add(hours <= MAX_WEEKLY_HOURS)
            # ≥40h soft:
            f40 = model.NewBoolVar(f"h40_{n}_{w}")
            model.Add(hours >= 40).OnlyEnforceIf(f40)
            model.Add(hours <  40).OnlyEnforceIf(f40.Not())
            penalty_terms.append((1 - f40) * PEN_HOURS40)
            # ≥30h soft:
            f30 = model.NewBoolVar(f"h30_{n}_{w}")
            model.Add(hours >= 30).OnlyEnforceIf(f30)
            model.Add(hours <  30).OnlyEnforceIf(f30.Not())
            penalty_terms.append((1 - f30) * PEN_HOURS30)

    # === E) One shift/day soft ===
    for n in nurse_names:
        for d in range(num_days):
            sw = sum(work[n,d,s] for s in range(3))
            dbl= model.NewBoolVar(f"dbl_{n}_{d}")
            model.Add(sw <= 1).OnlyEnforceIf(dbl.Not())
            model.Add(sw >  1).OnlyEnforceIf(dbl)
            penalty_terms.append(dbl * PEN_DOUBLE_SHIFT)

    # === F) Coverage per shift: seniors≥1 hard, ≥4 nurses soft (≥3 hard fallback) ===
    for d in range(num_days):
        for s in range(3):
            # ≥1 senior hard
            model.Add(sum(work[n,d,s] for n in senior_names) >= MIN_SENIORS_PER_SHIFT)
            # ≥3 total hard fallback
            cov = sum(work[n,d,s] for n in nurse_names)
            model.Add(cov >= MIN_NURSES_PER_SHIFT - 1)
            # ≥4 soft
            u4 = model.NewBoolVar(f"u4_{d}_{s}")
            model.Add(cov >= MIN_NURSES_PER_SHIFT).OnlyEnforceIf(u4.Not())
            model.Add(cov <  MIN_NURSES_PER_SHIFT).OnlyEnforceIf(u4)
            penalty_terms.append(u4 * PEN_UNDER4)

    # === Medium: AM coverage / senior‐AM with 3‐level fallback & hard “AM>others” ===
    for d in range(num_days):
        total = sum(work[n,d,s] for n in nurse_names for s in range(3))
        am    = sum(work[n,d,0] for n in nurse_names)
        pm    = sum(work[n,d,1] for n in nurse_names)
        nt    = sum(work[n,d,2] for n in nurse_names)
        am_s  = sum(work[n,d,0] for n in senior_names)
        pm_s  = sum(work[n,d,1] for n in senior_names)
        nt_s  = sum(work[n,d,2] for n in senior_names)

        pct1,pct2,pct3 = AM_COVERAGE_MIN_PERCENT, AM_COVERAGE_MIN_PERCENT-10, AM_COVERAGE_MIN_PERCENT-20

        # AM% levels
        for pct,pen,tag in [(pct1,PEN_AM60,60),(pct2,PEN_AM60//2,50),(pct3,PEN_AM60//5,40)]:
            lvl = model.NewBoolVar(f"am{tag}_{d}")
            model.Add(am*100 >= pct*total).OnlyEnforceIf(lvl)
            model.Add(am*100 <  pct*total).OnlyEnforceIf(lvl.Not())
            penalty_terms.append((1-lvl)*pen)
        # hard fallback: AM>PM & AM>NT
        model.Add(am > pm)
        model.Add(am > nt)

        # senior‐AM% levels
        for pct,pen,tag in [(pct1,PEN_AM60_SENIORS,60),(pct2,PEN_AM60_SENIORS//2,50),(pct3,PEN_AM60_SENIORS//5,40)]:
            sl = model.NewBoolVar(f"amsen{tag}_{d}")
            model.Add(am_s*100 >= pct*am).OnlyEnforceIf(sl)
            model.Add(am_s*100 <  pct*am).OnlyEnforceIf(sl.Not())
            penalty_terms.append((1-sl)*pen)
        # hard fallback: senior‐AM > senior‐PM & senior‐NT
        model.Add(am_s > pm_s)
        model.Add(am_s > nt_s)

    # === Weekend‐rest soft ===
    for n in nurse_names:
        for d1,d2 in weekend_days:
            if d1+7 < num_days:
                for s in range(3):
                    v = model.NewBoolVar(f"wr_{n}_{d1}_{s}")
                    model.AddBoolOr([
                        work[n,d1,s].Not(),
                        work[n,d1+7,s].Not(),
                        v
                    ])
                    penalty_terms.append(v * PEN_WEEKEND_REST)

    # === Preferences & fairness ===
    for n in nurse_names:
        sat_list = []
        for d,s in shift_prefs[n].items():
            sat = model.NewBoolVar(f"pref_{n}_{d}")
            model.Add(work[n,d,s] == 1).OnlyEnforceIf(sat)
            model.Add(work[n,d,s] == 0).OnlyEnforceIf(sat.Not())
            sat_list.append(sat)
            penalty_terms.append((1-sat)*PEN_PREF_UNMET)
        total_sat[n] = sum(sat_list)

    # fairness gap
    mn = model.NewIntVar(0,num_days,"min_sat")
    mx = model.NewIntVar(0,num_days,"max_sat")
    model.AddMinEquality(mn,list(total_sat.values()))
    model.AddMaxEquality(mx,list(total_sat.values()))
    gap = model.NewIntVar(0,num_days,"gap")
    model.Add(gap == mx - mn)
    penalty_terms.append(gap * PEN_FAIRNESS_GAP)

    # === Objective & Solve ===
    model.Minimize(sum(penalty_terms))
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 60
    status = solver.Solve(model)
    # never raise—even if INFEASIBLE / UNKNOWN, we extract the “best” assignment

    # === Extract schedule & summary ===
    dates  = [start_date + timedelta(days=i) for i in range(num_days)]
    cols   = [d.strftime('%a %Y-%m-%d') for d in dates]
    schedule, summary = {}, []

    for n in nurse_names:
        row, h1, h2, cnt, met, unmet = [], 0, 0, [0,0,0], 0, []
        for d in range(num_days):
            # collect assigned
            assigned = [s for s in range(3) if solver.Value(work[n,d,s])]
            if   d in mc_days[n]:       cell = "MC"
            elif not assigned:          cell = "Rest"
            elif len(assigned)==1:      cell = SHIFT_LABELS[assigned[0]]
            else:                       cell = "/".join(SHIFT_LABELS[s] for s in assigned)+"*"
            row.append(cell)
            # track hours & counts
            for s in assigned:
                h = SHIFT_HOURS[s]
                if d < 7:
                    h1 += h
                else:
                    h2 += h
                cnt[s] += 1
            # preferences
            if d in shift_prefs[n]:
                if shift_prefs[n][d] in assigned:
                    met += 1
                else:
                    unmet.append(dates[d].strftime('%a %Y-%m-%d'))

        schedule[n] = row
        summary.append({
            'Nurse': n,
            'Hours_Week1': h1,
            'Hours_Week2': h2,
            'AM': cnt[0], 'PM': cnt[1], 'Night': cnt[2],
            'Rest': row.count("Rest"),
            'MC_Days': len(mc_days[n]),
            'Prefs_Met': met,
            'Prefs_Unmet': len(unmet),
            'Unmet_Days': "; ".join(unmet)
        })

    return pd.DataFrame.from_dict(schedule, orient='index', columns=cols), pd.DataFrame(summary)

In [286]:
profiles_df   = load_nurse_profiles("nurse_profiles.xlsx")
preferences_df = load_shift_preferences("nurse_preferences.xlsx")

# Validate the loaded data
validate_nurse_data(profiles_df, preferences_df)

# Schedule generation parameters
start_date = preferences_df.columns[0]
num_days = len(preferences_df.columns)

# Build schedule and summary
df_schedule, df_summary = build_schedule_model(
    profiles_df, preferences_df, start_date, num_days
)

# Export results
df_schedule.to_excel("nurse_schedule.xlsx", sheet_name="Schedule", index=True)
df_summary.to_excel("nurse_summary.xlsx", sheet_name="Summary", index=False)
df_schedule.to_csv("nurse_schedule.csv")
df_summary.to_csv("nurse_summary.csv")

# Confirmation
print("✅ Files generated.")


✅ Nurse profile and preference names match.
✅ Files generated.
